
# 📰 Task 1 — RPP News Retrieval with LangChain + Chroma

Este notebook construye un buscador semántico de noticias de **RPP Perú** siguiendo la rúbrica:

**Load → Tokenize → Embed → Store → Query**


## 0) Instalación de dependencias

In [4]:

!pip -q install feedparser tiktoken sentence-transformers langchain langchain-community langchain-chroma pandas matplotlib


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1) Imports y configuración

In [5]:

import os
from typing import List, Dict, Any

import feedparser
import pandas as pd
import tiktoken

from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from langchain_chroma import Chroma

os.makedirs("outputs", exist_ok=True)
print("Entorno listo ✅")


Entorno listo ✅


## 2) Parámetros generales

In [6]:

RSS_URL = "https://rpp.pe/rss"
N_ITEMS = 50
MAX_TOKENS = 800
ENCODING = "cl100k_base"


## 3) Funciones de tokenización y chunking

In [7]:

enc = tiktoken.get_encoding(ENCODING)

def count_tokens(text: str) -> int:
    return len(enc.encode(text or ""))

def chunk_text(text: str, max_tokens: int = MAX_TOKENS) -> List[str]:
    tokens = enc.encode(text or "")
    if len(tokens) <= max_tokens:
        return [text]
    chunks, step = [], max_tokens
    for i in range(0, len(tokens), step):
        chunk_tokens = tokens[i:i+step]
        chunks.append(enc.decode(chunk_tokens))
    return chunks


## 4) Cargar noticias desde el RSS de RPP

In [8]:

def load_rss_rpp(rss_url: str = RSS_URL, limit: int = N_ITEMS) -> pd.DataFrame:
    feed = feedparser.parse(rss_url)
    items = []
    for entry in feed.entries[:limit]:
        items.append({
            "title": entry.get("title", ""),
            "description": entry.get("summary", ""),
            "link": entry.get("link", ""),
            "date_published": entry.get("published", "")
        })
    return pd.DataFrame(items, columns=["title","description","link","date_published"])

df_raw = load_rss_rpp()
print(f"Noticias cargadas: {len(df_raw)}")
df_raw.head(10)


Noticias cargadas: 50


,title,description,link,date_published
0,Arequipa: hombre que agredió brutalmente a su ...,Pese a que la agresión fue registrada en video...,https://rpp.pe/peru/arequipa/arequipa-hombre-q...,"Sat, 18 Oct 2025 22:53:43 -0500"
1,Papa León XIV bendecirá este domingo la imagen...,La gran procesión es organizada por la hermand...,https://rpp.pe/mundo/actualidad/papa-leon-xiv-...,"Sat, 18 Oct 2025 21:56:57 -0500"
2,Temblor en Chile hoy 18 de octubre: Epicentro ...,¿Cuál fue el último Temblor en Chile hoy 18 de...,https://rpp.pe/mundo/chile/temblor-en-chile-ho...,"Fri, 17 Oct 2025 20:09:45 -0500"
3,"Partidos de hoy, domingo 19 de octubre del 202...",Fútbol EN VIVO | Horarios y canales de TV para...,https://rpp.pe/futbol/futbol-mundial/partidos-...,"Sat, 18 Oct 2025 22:50:29 -0500"
4,Cúal fue el último temblor en México hoy 18 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Thu, 16 Oct 2025 06:48:35 -0500"
5,Universitario se topó con una muralla: cayó 3-...,Universitario perdió 3-0 ante Gimnasia en la d...,https://rpp.pe/voley/mas-voley/universitario-v...,"Sat, 18 Oct 2025 22:15:03 -0500"
6,Liga MX: CF Monterrey logró sacar el empate de...,CF Monterrey vs Pumas UNAM se enfrentaron por ...,https://rpp.pe/futbol/ligas-internacionales/li...,"Sat, 18 Oct 2025 22:20:07 -0500"
7,Liga MX: Derrota de Mazatlán por 2-0 ante Chivas,Chivas vs Mazatlán se enfrentaron por la Méxic...,https://rpp.pe/futbol/ligas-internacionales/li...,"Sat, 18 Oct 2025 22:15:12 -0500"
8,Universitario vs Ayacucho FC VIVO: ¿a qué hora...,Universitario recibe a Ayacucho FC por la fech...,https://rpp.pe/futbol/descentralizado/universi...,"Sat, 18 Oct 2025 21:40:07 -0500"
9,Por todo lo alto: Regatas Lima venció 3-1 al C...,Regatas Lima venció 3-1 al Circolo Sportivo It...,https://rpp.pe/voley/mas-voley/regatas-vs-circ...,"Sat, 18 Oct 2025 21:30:34 -0500"


## 5) Construcción de documentos y embeddings

In [9]:

embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

class STEmbeddings(Embeddings):
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return embed_model.encode(texts, normalize_embeddings=True).tolist()
    def embed_query(self, text: str) -> List[float]:
        return embed_model.encode([text], normalize_embeddings=True).tolist()[0]

def build_docs(df: pd.DataFrame) -> List[Dict[str, Any]]:
    docs = []
    for i, row in df.iterrows():
        combined_text = (row["title"] or "") + " | " + (row["description"] or "")
        pieces = chunk_text(combined_text, max_tokens=MAX_TOKENS)
        for j, piece in enumerate(pieces):
            docs.append({
                "id": f"rpp-{i}-{j}",
                "text": piece,
                "metadata": {
                    "title": row["title"],
                    "description": row["description"],
                    "link": row["link"],
                    "date_published": row["date_published"]
                }
            })
    return docs

docs = build_docs(df_raw)
len(docs)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

50

## 6) Crear colección Chroma con LangChain

In [10]:

def build_vectorstore_langchain(docs: List[Dict[str, Any]], collection_name="rpp_news_langchain") -> Chroma:
    vectorstore = Chroma(collection_name=collection_name, embedding_function=STEmbeddings())
    vectorstore.add_texts(
        texts=[d["text"] for d in docs],
        metadatas=[d["metadata"] for d in docs],
        ids=[d["id"] for d in docs]
    )
    return vectorstore

def build_retriever(vectorstore: Chroma, k: int = 8):
    return vectorstore.as_retriever(search_kwargs={"k": k})

vectorstore = build_vectorstore_langchain(docs)
retriever = build_retriever(vectorstore, k=8)
print("Documentos indexados:", len(docs))


Documentos indexados: 50


## 7) Consulta de ejemplo

In [11]:

def query_news(retriever, query: str, k: int = 8) -> pd.DataFrame:
    docs_found = retriever.get_relevant_documents(query)
    rows = []
    for d in docs_found:
        m = d.metadata or {}
        rows.append({
            "title": m.get("title",""),
            "description": m.get("description",""),
            "link": m.get("link",""),
            "date_published": m.get("date_published","")
        })
    return pd.DataFrame(rows, columns=["title","description","link","date_published"])

df_results = query_news(retriever, "Últimas noticias de economía", k=8)
df_results


/tmp/ipython-input-4065674914.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_found = retriever.get_relevant_documents(query)


,title,description,link,date_published
0,Arequipa: hombre que agredió brutalmente a su ...,Pese a que la agresión fue registrada en video...,https://rpp.pe/peru/arequipa/arequipa-hombre-q...,"Sat, 18 Oct 2025 22:53:43 -0500"
1,Ministro del Interior: “Seremos implacables co...,El ministro Vicente Tiburcio participó en la c...,https://rpp.pe/politica/gobierno/ministro-del-...,"Sat, 18 Oct 2025 20:48:38 -0500"
2,Acorta distancia: Cusco FC venció 2-1 a Cienci...,Cusco FC venció 2-1 a Cienciano y se puso a cu...,https://rpp.pe/futbol/descentralizado/ciencian...,"Sat, 18 Oct 2025 20:00:07 -0500"
3,Detienen a tres policías acusados de cobrar co...,"Según el Ministerio Público, el chofer fue coa...",https://rpp.pe/lima/policiales/detienen-a-tres...,"Sat, 18 Oct 2025 16:58:57 -0500"
4,Universitario se topó con una muralla: cayó 3-...,Universitario perdió 3-0 ante Gimnasia en la d...,https://rpp.pe/voley/mas-voley/universitario-v...,"Sat, 18 Oct 2025 22:15:03 -0500"
5,Melania Urbina se quiebra al despedirse de su ...,La actriz compartió un emotivo video en redes ...,https://rpp.pe/famosos/farandula/melania-urbin...,"Sat, 18 Oct 2025 18:39:53 -0500"
6,Venezolanos piden en Roma liberación de presos...,"El pasado 29 de septiembre, la líder opositora...",https://rpp.pe/mundo/actualidad/venezolanos-pi...,"Sat, 18 Oct 2025 20:02:13 -0500"
7,Brasil: Al menos 17 muertos y una veintena de ...,"Según la investigación preliminar, el conducto...",https://rpp.pe/mundo/actualidad/brasil-al-meno...,"Sat, 18 Oct 2025 19:45:57 -0500"


## 8) Guardar resultados en CSV

In [12]:

out_path = "outputs/query_economia.csv"
df_results.to_csv(out_path, index=False)
print("CSV guardado en:", out_path)


CSV guardado en: outputs/query_economia.csv


## 9) Pipeline completo end-to-end

In [13]:

def pipeline_end_to_end(query="Últimas noticias de política", k=6) -> pd.DataFrame:
    df = load_rss_rpp()
    docs = build_docs(df)
    vs = build_vectorstore_langchain(docs, collection_name="rpp_news_pipeline")
    ret = build_retriever(vs, k=k)
    return query_news(ret, query, k=k)

df_pipeline = pipeline_end_to_end()
df_pipeline


,title,description,link,date_published
0,Detienen a tres policías acusados de cobrar co...,"Según el Ministerio Público, el chofer fue coa...",https://rpp.pe/lima/policiales/detienen-a-tres...,"Sat, 18 Oct 2025 16:58:57 -0500"
1,Ministro del Interior: “Seremos implacables co...,El ministro Vicente Tiburcio participó en la c...,https://rpp.pe/politica/gobierno/ministro-del-...,"Sat, 18 Oct 2025 20:48:38 -0500"
2,Arequipa: hombre que agredió brutalmente a su ...,Pese a que la agresión fue registrada en video...,https://rpp.pe/peru/arequipa/arequipa-hombre-q...,"Sat, 18 Oct 2025 22:53:43 -0500"
3,Melania Urbina se quiebra al despedirse de su ...,La actriz compartió un emotivo video en redes ...,https://rpp.pe/famosos/farandula/melania-urbin...,"Sat, 18 Oct 2025 18:39:53 -0500"
4,Venezolanos piden en Roma liberación de presos...,"El pasado 29 de septiembre, la líder opositora...",https://rpp.pe/mundo/actualidad/venezolanos-pi...,"Sat, 18 Oct 2025 20:02:13 -0500"
5,La Molina: adulta mayor pide ayuda para conseg...,"Rotafono de RPP | Según Judith López, desde el...",https://rpp.pe/rotafono/servicios-publicos/la-...,"Sat, 18 Oct 2025 20:12:33 -0500"



## ✅ Checklist de rúbrica
- RSS parseado (RPP)  
- Tokenización con `tiktoken`  
- Embeddings con `all-MiniLM-L6-v2`  
- ChromaDB con LangChain (store + upsert + retrieval)  
- Orquestación modular  
- Tabla con 4 columnas (title, description, link, date_published)  
- CSV en `outputs/`  
